In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json 
import requests
import folium


In [3]:
# Define the URL for the free bike status feed
url = "https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/free_bike_status.json"

# Fetch the data
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    free_bike_data = response.json()
    # Extract the 'bikes' data into a DataFrame
    bikes = free_bike_data['data']['bikes']
    df = pd.DataFrame(bikes)
    
    # Display the first few rows of the DataFrame
    print("Data successfully retrieved!")
    print(df.head())
else:
    print(f"Failed to fetch data. HTTP Status Code: {response.status_code}")

Data successfully retrieved!
   is_reserved        lon  current_range_meters  \
0            0 -77.062423            29290.0608   
1            0 -76.987389            27036.9792   
2            0 -77.110925            16254.3744   
3            0 -76.985159            31060.3392   
4            0 -77.097236            13035.6864   

                                         rental_uris vehicle_type_id  \
0  {'android': 'https://dc.lft.to/lastmile_qr_sca...               2   
1  {'android': 'https://dc.lft.to/lastmile_qr_sca...               2   
2  {'android': 'https://dc.lft.to/lastmile_qr_sca...               2   
3  {'android': 'https://dc.lft.to/lastmile_qr_sca...               2   
4  {'android': 'https://dc.lft.to/lastmile_qr_sca...               2   

                            bike_id  is_disabled        lat  
0  47975f44ac4dfe7e219d337d38ece106            0  38.821637  
1  feba6ff94ef6b2448b995607946e40d6            0  38.958191  
2  c5017034ed24aaff07d1b760bbc37e59          

In [6]:
# Define the URL for the free bike status feed
url = "https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/free_bike_status.json"

# Fetch the data
response = requests.get(url)
if response.status_code == 200:
    free_bike_data = response.json()
    bikes = free_bike_data['data']['bikes']
    df = pd.DataFrame(bikes)
else:
    print(f"Failed to fetch data. HTTP Status Code: {response.status_code}")
    df = pd.DataFrame()  # Empty DataFrame for safety

# Ensure the DataFrame is not empty
if not df.empty:
    # Create a map centered at the average location of all bikes
    map_center = [df['lat'].mean(), df['lon'].mean()]
    bike_map = folium.Map(location=map_center, zoom_start=13)

    # Add markers for each bike
    for _, bike in df.iterrows():
        folium.Marker(
            location=[bike['lat'], bike['lon']],
            popup=f"Bike ID: {bike['bike_id']}\nDisabled: {bike['is_disabled']}",
            icon=folium.Icon(color="green" if bike['is_disabled'] == 0 else "red"),
        ).add_to(bike_map)

    # Save and display the map
    bike_map.save("free_bike_status_map.html")
    bike_map
else:
    print("No data available to map.")

# Save to CSV
df.to_csv("free_bike_status.csv", index=False)
print("Data successfully exported as 'free_bike_status.csv'.")

Data successfully exported as 'free_bike_status.csv'.


In [6]:
import requests
import pandas as pd

# Define the URL for the vehicle_types.json feed
url = "https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/vehicle_types.json"

# Fetch the data
response = requests.get(url)

if response.status_code == 200:
    # Parse the JSON response
    vehicle_data = response.json()
    
    # Extract the vehicle types
    vehicle_types = vehicle_data['data']['vehicle_types']
    
    # Convert the data into a DataFrame for better visualization
    df = pd.DataFrame(vehicle_types)
    
    # Display the first few rows of the DataFrame
    print("Full Vehicle Type Data:")
    print(df.head())  # Display the first 5 rows
else:
    print(f"Failed to fetch data. HTTP Status Code: {response.status_code}")

Full Vehicle Type Data:
  form_factor vehicle_type_id  propulsion_type  max_range_meters
0     bicycle               1            human               NaN
1     bicycle               2  electric_assist          55844.24


In [11]:
import requests
import pandas as pd

# Fetch JSON data from API
url = "https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/station_status.json"
response = requests.get(url)

if response.status_code == 200:
    json_data = response.json()  # Convert response to dictionary
    
    # Extract the "stations" key
    stations = json_data["data"]["stations"]
    
    # Flatten nested 'vehicle_types_available'
    flat_data = []
    for station in stations:
        base_info = {
            "station_id": station.get("station_id"),
            "num_bikes_available": station.get("num_bikes_available", 0),
            "num_bikes_disabled": station.get("num_bikes_disabled", 0),
            "num_docks_available": station.get("num_docks_available", 0),
            "num_docks_disabled": station.get("num_docks_disabled", 0),
            "num_ebikes_available": station.get("num_ebikes_available", 0),
            "num_scooters_available": station.get("num_scooters_available", 0),
            "num_scooters_unavailable": station.get("num_scooters_unavailable", 0),
            "is_installed": station.get("is_installed", 0),
            "is_renting": station.get("is_renting", 0),
            "is_returning": station.get("is_returning", 0),
            "last_reported": station.get("last_reported", 0)
        }
        
        # Extract vehicle types if available
        vehicle_types = station.get("vehicle_types_available", [])
        if vehicle_types:
            for vehicle in vehicle_types:
                vehicle_info = base_info.copy()
                vehicle_info["vehicle_type_id"] = vehicle.get("vehicle_type_id")
                vehicle_info["vehicle_count"] = vehicle.get("count")
                flat_data.append(vehicle_info)
        else:
            base_info["vehicle_type_id"] = None
            base_info["vehicle_count"] = None
            flat_data.append(base_info)

    # Convert to DataFrame
    stations = pd.DataFrame(flat_data)
    
    # Display first few rows
    print(stations.head())

    # Save to CSV (optional)
    stations.to_csv("station_status.csv", index=False)
    print("Data saved to 'station_status.csv'.")

else:
    print(f"Failed to fetch data: {response.status_code}")

                             station_id  num_bikes_available  \
0  fbbeab20-556d-4275-98a9-f84d316c157f                    9   
1  fbbeab20-556d-4275-98a9-f84d316c157f                    9   
2  08251448-1f3f-11e7-bf6b-3863bb334450                    1   
3  08251448-1f3f-11e7-bf6b-3863bb334450                    1   
4                   1892760823152249488                   10   

   num_bikes_disabled  num_docks_available  num_docks_disabled  \
0                   1                    9                   0   
1                   1                    9                   0   
2                   0                   18                   0   
3                   0                   18                   0   
4                   0                    1                   0   

   num_ebikes_available  num_scooters_available  num_scooters_unavailable  \
0                     0                       0                         0   
1                     0                       0                 